In [0]:
%sql

create schema if not exists rwd.bronze;
     

In [0]:
source_path = "/Volumes/rwd/bronze/myvolume/bronze/staging/primary_cancer_condition/"
checkpoint_path = "/Volumes/rwd/bronze/myvolume/bronze/primary_cancer_condition_raw/checkpoint/"
schema_location = "/Volumes/rwd/bronze/myvolume/bronze/primary_cancer_condition_raw/schema/"


In [0]:
# Autoloader read
df = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .option("cloudFiles.maxFilesPerTrigger", 1)
          .option("cloudFiles.schemaLocation", schema_location)  
          .load(source_path)
     )

In [0]:
# Write Bronze table (append)
(df.drop("_rescued_data")
    .writeStream
    .format("delta")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")
    .outputMode("append")
    .trigger(availableNow=True)
    .toTable("rwd.bronze.primary_cancer_condition_raw")
)

In [0]:
%sql

select * from rwd.bronze.primary_cancer_condition_raw

Patient_ID,Primary_Cancer_Condition,Diagnosis_Date,Hospital
P001,Breast Carcinoma,2024-08-12,Apollo Hospitals
P002,Lung Adenocarcinoma,2023-11-03,AIIMS Delhi
P003,Colorectal Cancer,2024-05-19,Fortis Chennai
P004,Prostate Cancer,2023-09-25,CMC Vellore
P005,Hepatocellular Carcinoma,2024-07-14,Narayana Health
P006,Ovarian Cancer,2023-12-01,Max Healthcare
P007,Melanoma,2024-06-22,KIMS Hyderabad
P008,Pancreatic Cancer,2024-03-18,Manipal Hospitals
P009,Gastric Cancer,2024-04-10,Sankara Nethralaya
P010,Bladder Cancer,2023-10-05,Tata Memorial


In [0]:
schema_df = spark.read.json("/Volumes/rwd/bronze/myvolume/bronze/primary_cancer_condition_raw/schema/*")
display(schema_df)


_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Patient_ID"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Primary_Cancer_Condition"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Diagnosis_Date"",""type"":""date"",""nullable"":true,""metadata"":{""__detected_date_formats"":""yyyy-M-d""}},{""name"":""Hospital"",""type"":""string"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[]}"


In [0]:
%sql
MERGE INTO rwd.bronze.primary_cancer_condition AS target
USING rwd.bronze.primary_cancer_condition_raw AS source
ON target.patient_id = source.patient_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
select * from rwd.bronze.primary_cancer_condition

Patient_ID,Primary_Cancer_Condition,Diagnosis_Date,Hospital
P001,Breast Carcinoma,2024-08-12,Apollo Hospitals
P002,Lung Adenocarcinoma,2023-11-03,AIIMS Delhi
P003,Colorectal Cancer,2024-05-19,Fortis Chennai
P004,Prostate Cancer,2023-09-25,CMC Vellore
P005,Hepatocellular Carcinoma,2024-07-14,Narayana Health
P006,Ovarian Cancer,2023-12-01,Max Healthcare
P007,Melanoma,2024-06-22,KIMS Hyderabad
P008,Pancreatic Cancer,2024-03-18,Manipal Hospitals
P009,Gastric Cancer,2024-04-10,Sankara Nethralaya
P010,Bladder Cancer,2023-10-05,Tata Memorial
